In [1]:
import nltk
import numpy as np

from nltk.corpus import reuters
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

import np_utils
import keras

documents = reuters.fileids()

train_docs = filter(lambda doc: doc.startswith("train"),documents);
test_docs = filter(lambda doc: doc.startswith("test"),documents);

sentences = [reuters.raw(document) for document in train_docs]


In [6]:
len(sentences)

7769

In [5]:
import string

words = [list(reuters.words(article)) for article in train_docs]
print(len(words))
stop_words = set(stopwords.words('english'))

from nltk.stem.porter import PorterStemmer
stmr = PorterStemmer()
k = 0
words_out = []
for sentence in words:
    good_sentence = []
    for word in sentence:
        if word in stop_words or word in string.punctuation:
            pass
        else:
            good_sentence.append(stmr.stem(word))
    good_sentence = ' '.join(good_sentence)
    words_out.append(good_sentence)
    

7769


In [77]:
def test_architecture(nb_classes, nb_tokens, maxlen, embed_l2=1E-6, return_attention=False):

    # define embedding layer that turns word tokens into vectors
    # an activation function is used to bound the values of the embedding

    model_input = Input(shape=(maxlen,), dtype='int32', name= 'Input First')

    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None

    embed = Embedding(input_dim=nb_tokens,
                      output_dim=256,
                      mask_zero=True,
                      input_length=maxlen,
                      embeddings_regularizer=embed_reg,
                      name='embedding')

    x = embed(model_input)

    x = Activation('tanh')(x)

    x = Bidirectional(LSTM(512, return_sequences=True), name="bi_lstm_0")(x)

    weights = None
    x = AttentionWeightedAverage(name='attlayer', return_attention=return_attention)(x)

    outputs = [x]

    return Model(inputs=[model_input], outputs=outputs, name="riminder")

In [87]:
model3 = test_architecture(nb_classes,nb_token,maxlen)

model4 = Model

Beggining build model
Embedding reg
Beggining embedding layer
embeb finished
x finished
Finish introduction
LSTM building
LSTMs ready
Attention layer ready


In [17]:
import json

from keras.models import Model, Sequential
from keras.layers.merge import concatenate
from keras.layers import Input, Bidirectional, Embedding, Dense, Dropout, SpatialDropout1D, LSTM, Activation
from keras.regularizers import L1L2
from deepmoji.attlayer import AttentionWeightedAverage
print('Library imported')

## Building of the vocabulary
from deepmoji.sentence_tokenizer import SentenceTokenizer
from deepmoji.model_def import deepmoji_feature_encoding
from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH

maxlen = 100

print('Tokenizing using dictionary from {}'.format(VOCAB_PATH))
with open(VOCAB_PATH, 'r') as f:
    vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, maxlen)
tokenized, _, _ = st.tokenize_sentences(words_out)

nb_classes = 90

batch_size = 64

nb_token = tokenized.max()+1

# model = test_architecture(nb_classes,nb_token,maxlen)


Library imported
Tokenizing using dictionary from C:\Users\victor\Documents\GitHub\riminder/model/vocabulary.json


In [26]:
## training
categories = [reuters.categories(document) for document in train_docs]
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
encoded_cat =  mlb.fit_transform(categories)

In [9]:
model.compile(optimizer = "Adam",loss = "categorical_crossentropy")
model.fit(tokenized,encoded_cat,batch_size = 100,epochs = 10)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '3288' (I am process '7152')


Epoch 1/10
7769/7769 [==============================] - 478s 61ms/step - loss: 3.3198
Epoch 2/10
7769/7769 [==============================] - 479s 62ms/step - loss: 2.4460
Epoch 3/10
7769/7769 [==============================] - 480s 62ms/step - loss: 2.0239
Epoch 4/10
7769/7769 [==============================] - 504s 65ms/step - loss: 1.6680
Epoch 5/10
7769/7769 [==============================] - 457s 59ms/step - loss: 1.3257
Epoch 6/10
7769/7769 [==============================] - 436s 56ms/step - loss: 1.1363
Epoch 7/10
7769/7769 [==============================] - 446s 57ms/step - loss: 1.0354
Epoch 8/10
7769/7769 [==============================] - 463s 60ms/step - loss: 0.9391
Epoch 9/10
7769/7769 [==============================] - 463s 60ms/step - loss: 0.8798
Epoch 10/10
7769/7769 [==============================] - 441s 57ms/step - loss: 0.7999


In [14]:
model = load_model('model_riminder_good.h5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage})

In [18]:
# Processing of the test data set
words_test = [list(reuters.words(article)) for article in test_docs]
print(len(words_test))
stop_words = set(stopwords.words('english'))

from nltk.stem.porter import PorterStemmer
stmr = PorterStemmer()
k = 0
words_out_test = []
for sentence in words_test:
    good_sentence = []
    for word in sentence:
        if word in stop_words or word in string.punctuation:
            pass
        else:
            good_sentence.append(stmr.stem(word))
    good_sentence = ' '.join(good_sentence)
    words_out_test.append(good_sentence)

tokenized_test, _, _ = st.tokenize_sentences(words_out_test)   

In [27]:
prob = model.predict(tokenized_test)
categories_test = [reuters.categories(document) for document in test_docs]
encoded_cat_test =  mlb.fit_transform(categories)

In [43]:
prob_util = prob.copy()

In [44]:
category = reuters.categories(documents)

def output(result,real):
    print('Classe cible --> {}'.format(real))
    result = list(result.copy())
    for k in range(4):
        index_max = np.argmax(result)
        maximum = result.pop(index_max)
        print('La {}ème classe est  {} avec la proba {}'.format(k,category[index_max],maximum))

for k in range(prob_util.shape[0]):
    output(prob_util[k],categories_test[k])

Classe cible --> [u'trade']
La 0ème classe est  trade avec la proba 0.794047892094
La 1ème classe est  iron-steel avec la proba 0.0354652777314
La 2ème classe est  grain avec la proba 0.0321452692151
La 3ème classe est  tin avec la proba 0.0151963429525
Classe cible --> [u'grain']
La 0ème classe est  grain avec la proba 0.290636837482
La 1ème classe est  oat avec la proba 0.113799504936
La 2ème classe est  corn avec la proba 0.105080142617
La 3ème classe est  rice avec la proba 0.103961750865
Classe cible --> [u'crude', u'nat-gas']
La 0ème classe est  crude avec la proba 0.537243425846
La 1ème classe est  gas avec la proba 0.0842238888144
La 2ème classe est  cpi avec la proba 0.0730573609471
La 3ème classe est  money-fx avec la proba 0.0615142919123
Classe cible --> [u'corn', u'grain', u'rice', u'rubber', u'sugar', u'tin', u'trade']
La 0ème classe est  grain avec la proba 0.213414520025
La 1ème classe est  veg-oil avec la proba 0.142696842551
La 2ème classe est  tin avec la proba 0.126

La 1ème classe est  crude avec la proba 0.264580279589
La 2ème classe est  fuel avec la proba 0.0224926583469
La 3ème classe est  acq avec la proba 0.0173503868282
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999877810478
La 1ème classe est  housing avec la proba 3.99419877795e-05
La 2ème classe est  rapeseed avec la proba 3.13361233566e-05
La 3ème classe est  acq avec la proba 1.14770155051e-05
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999404370785
La 1ème classe est  acq avec la proba 0.000505882373545
La 2ème classe est  palm-oil avec la proba 2.75070869975e-05
La 3ème classe est  acq avec la proba 2.38477550738e-05
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999981224537
La 1ème classe est  acq avec la proba 1.39126732392e-05
La 2ème classe est  hog avec la proba 1.62870378517e-06
La 3ème classe est  palladium avec la proba 7.13184249435e-07
Classe cible --> [u'interest', u'money-fx']
La 0ème classe est  inter

La 3ème classe est  heat avec la proba 0.000141975775477
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999196171761
La 1ème classe est  acq avec la proba 0.00074787781341
La 2ème classe est  palm-oil avec la proba 1.80875395017e-05
La 3ème classe est  acq avec la proba 1.307835646e-05
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999709963799
La 1ème classe est  acq avec la proba 0.000237258485868
La 2ème classe est  palm-oil avec la proba 1.52369448188e-05
La 3ème classe est  acq avec la proba 1.27498742586e-05
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999644398689
La 1ème classe est  acq avec la proba 0.000309710972942
La 2ème classe est  palm-oil avec la proba 1.43563183883e-05
La 3ème classe est  acq avec la proba 1.21013208627e-05
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.99967867136
La 1ème classe est  acq avec la proba 0.000252759404248
La 2ème classe est  acq avec la proba 1.48505505

La 0ème classe est  earn avec la proba 0.99891012907
La 1ème classe est  acq avec la proba 0.000936773780268
La 2ème classe est  acq avec la proba 5.38211024832e-05
La 3ème classe est  palladium avec la proba 4.5379911171e-05
Classe cible --> [u'acq']
La 0ème classe est  acq avec la proba 0.999000310898
La 1ème classe est  dmk avec la proba 0.000969587883446
La 2ème classe est  cpu avec la proba 6.08840991845e-06
La 3ème classe est  rand avec la proba 4.66352730655e-06
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999929368496
La 1ème classe est  housing avec la proba 2.51812034548e-05
La 2ème classe est  rapeseed avec la proba 1.64325592777e-05
La 3ème classe est  acq avec la proba 8.96571418707e-06
Classe cible --> [u'dlr', u'interest', u'money-fx']
La 0ème classe est  interest avec la proba 0.572251737118
La 1ème classe est  meal-feed avec la proba 0.354454755783
La 2ème classe est  gnp avec la proba 0.0394870229065
La 3ème classe est  dlr avec la proba 0.01674

La 1ème classe est  acq avec la proba 5.12637270731e-05
La 2ème classe est  hog avec la proba 1.19089982036e-05
La 3ème classe est  rand avec la proba 1.15138254841e-05
Classe cible --> [u'crude', u'nat-gas']
La 0ème classe est  crude avec la proba 0.615468502045
La 1ème classe est  naphtha avec la proba 0.328126549721
La 2ème classe est  fuel avec la proba 0.0162323694676
La 3ème classe est  groundnut avec la proba 0.0118689024821
Classe cible --> [u'earn']
La 0ème classe est  acq avec la proba 0.488431721926
La 1ème classe est  acq avec la proba 0.0805751979351
La 2ème classe est  dlr avec la proba 0.062643520534
La 3ème classe est  cpi avec la proba 0.0497737638652
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.986948549747
La 1ème classe est  acq avec la proba 0.00500610889867
La 2ème classe est  cpu avec la proba 0.0033750673756
La 3ème classe est  money-fx avec la proba 0.00227760244161
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.99987

...

Classe cible --> [u'crude', u'ship']
La 0ème classe est  crude avec la proba 0.705398917198
La 1ème classe est  rye avec la proba 0.1423060745
La 2ème classe est  naphtha avec la proba 0.0633736327291
La 3ème classe est  acq avec la proba 0.0287295803428
Classe cible --> [u'coffee']
La 0ème classe est  coffee avec la proba 0.912339150906
La 1ème classe est  rice avec la proba 0.0230932869017
La 2ème classe est  cocoa avec la proba 0.0155079448596
La 3ème classe est  sun-meal avec la proba 0.0132669126615
Classe cible --> [u'acq']
La 0ème classe est  acq avec la proba 0.999940097332
La 1ème classe est  dmk avec la proba 4.97305773024e-05
La 2ème classe est  lumber avec la proba 2.0041670723e-06
La 3ème classe est  rand avec la proba 1.6054864318e-06
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999769091606
La 1ème classe est  acq avec la proba 0.000121040480735
La 2ème classe est  acq avec la proba 3.23084605043e-05
La 3ème classe est  heat avec la proba 2.1089077

La 0ème classe est  earn avec la proba 0.999632716179
La 1ème classe est  acq avec la proba 0.000173137595993
La 2ème classe est  acq avec la proba 5.13597515237e-05
La 3ème classe est  heat avec la proba 4.22669218096e-05
Classe cible --> [u'interest', u'money-fx']
La 0ème classe est  trade avec la proba 0.666618525982
La 1ème classe est  money-fx avec la proba 0.263567894697
La 2ème classe est  dlr avec la proba 0.0232059434056
La 3ème classe est  instal-debt avec la proba 0.0226176325232
Classe cible --> [u'crude']
La 0ème classe est  ship avec la proba 0.401832967997
La 1ème classe est  crude avec la proba 0.264297515154
La 2ème classe est  tea avec la proba 0.200209915638
La 3ème classe est  acq avec la proba 0.0609969049692
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999976217747
La 1ème classe est  acq avec la proba 1.37288971018e-05
La 2ème classe est  hog avec la proba 3.57625003744e-06
La 3ème classe est  rape-oil avec la proba 1.3879822518e-06
Classe 

La 3ème classe est  iron-steel avec la proba 0.0457519106567
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999974012375
La 1ème classe est  acq avec la proba 9.45801912167e-06
La 2ème classe est  hog avec la proba 6.36439381196e-06
La 3ème classe est  rape-oil avec la proba 3.01477939502e-06
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.99922722578
La 1ème classe est  acq avec la proba 0.00068955385359
La 2ème classe est  acq avec la proba 1.92181396415e-05
La 3ème classe est  heat avec la proba 1.41007640195e-05
Classe cible --> [u'money-fx']
La 0ème classe est  gnp avec la proba 0.133673369884
La 1ème classe est  interest avec la proba 0.112545616925
La 2ème classe est  nzdlr avec la proba 0.0537350401282
La 3ème classe est  iron-steel avec la proba 0.0457519106567
Classe cible --> [u'crude']
La 0ème classe est  ship avec la proba 0.654702246189
La 1ème classe est  crude avec la proba 0.129044294357
La 2ème classe est  ipi avec la proba 0.1

La 3ème classe est  palm-oil avec la proba 9.29563611862e-05
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999473750591
La 1ème classe est  acq avec la proba 0.000468505750177
La 2ème classe est  palm-oil avec la proba 1.89882048289e-05
La 3ème classe est  acq avec la proba 1.47125456351e-05
Classe cible --> [u'acq']
La 0ème classe est  acq avec la proba 0.999859988689
La 1ème classe est  dmk avec la proba 7.44705539546e-05
La 2ème classe est  lumber avec la proba 1.16363744382e-05
La 3ème classe est  money-fx avec la proba 8.01998703537e-06
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.999895095825
La 1ème classe est  acq avec la proba 6.70050503686e-05
La 2ème classe est  hog avec la proba 8.93801188795e-06
La 3ème classe est  acq avec la proba 8.22118272481e-06
Classe cible --> [u'earn']
La 0ème classe est  earn avec la proba 0.99994289875
La 1ème classe est  acq avec la proba 2.15016862057e-05
La 2ème classe est  hog avec la proba 1.21308

La 1ème classe est  naphtha avec la proba 0.0376543775201
La 2ème classe est  fuel avec la proba 0.0082744769752
La 3ème classe est  groundnut avec la proba 0.00307176192291
Classe cible --> [u'carcass', u'hog', u'livestock']
La 0ème classe est  livestock avec la proba 0.514757096767
La 1ème classe est  carcass avec la proba 0.253279298544
La 2ème classe est  livestock avec la proba 0.038981705904
La 3ème classe est  heat avec la proba 0.0235469769686
Classe cible --> [u'gnp', u'grain', u'ipi']
La 0ème classe est  gnp avec la proba 0.647522747517
La 1ème classe est  interest avec la proba 0.125839531422
La 2ème classe est  iron-steel avec la proba 0.0377449020743
La 3ème classe est  nkr avec la proba 0.0188362635672
Classe cible --> [u'crude']
La 0ème classe est  crude avec la proba 0.714590013027
La 1ème classe est  rye avec la proba 0.1284109056
La 2ème classe est  naphtha avec la proba 0.0623996965587
La 3ème classe est  ipi avec la proba 0.0207379627973
Classe cible --> [u'ship']
L

In [102]:
#Cosine similarity
from nltk.stem.porter import PorterStemmer
from scipy.spatial.distance import cosine
import string

def pre_processing(sentence):
    good_sentence = []
    stop_words = set(stopwords.words('english'))
    stmr = PorterStemmer()
    for word in sentence:
         if word not in stop_words and word not in string.punctuation:
            good_sentence.append(stmr.stem(word))
    good_sentence = ' '.join(good_sentence)
    return(good_sentence)

def compute_cosine(code_doc_1,code_doc_2, embed = True):
    text_1,_,_ = st.tokenize_sentences([pre_processing(list(reuters.words(code_doc_1)))])
    text_2,_,_ = st.tokenize_sentences([pre_processing(list(reuters.words(code_doc_2)))])
    pred_1 = model.predict(text_1)
    pred_2 = model.predict(text_2)
    return(cosine(pred_1,pred_2),text_1)


In [60]:
a = 'test/14826'
b = 'test/14828'
compute_cosine(a,b)

0.84695546329021454

In [96]:
#Test, utilisation de la cosine similarity en sortie de la couche embedding
model_2 = Model(
        inputs=[model.layers[0].input], 
        outputs=[model.get_layer('embedding').output],
        name='similarity'
) 

In [110]:
pred = model_2.predict(compute_cosine(a,b)[1])

In [117]:
pred[0].shape

(100L, 256L)